### Notebook to engineer features  (a) Fraction of time temperature was above 90 F, (b) Fraction of time temperature was below 0 C and (c) Fraction of time PM2.5 was above EPA standards

In [1]:
## conda environment Weather_Prediction

import os
import numpy as np
import pandas as pd
import geopandas as gpd
import xarray as xr

In [2]:
PATH_1=os.path.join('../Weather_Data','CAMS','2010','2010_multi_level_26_variables.nc')
ml_26_variables = xr.open_dataset(PATH_1) ## multi level 26 variables
temperature=ml_26_variables['t'] ## get only temperature variable
temperature_squeezed=temperature.squeeze('model_level') ## remove model level dimension
temperature=temperature_squeezed.drop_vars('model_level') ## model becomes a feature after squeeze, so drop it
temperature

<xarray.DataArray 't' (valid_time: 2920, latitude: 34, longitude: 78)> Size: 31MB
[7743840 values with dtype=float32]
Coordinates:
  * valid_time  (valid_time) datetime64[ns] 23kB 2010-01-01 ... 2010-12-31T21...
  * latitude    (latitude) float64 272B 49.25 48.5 47.75 ... 26.0 25.25 24.5
  * longitude   (longitude) float64 624B 235.2 235.9 236.7 ... 291.4 292.1 292.9
Attributes: (12/31)
    GRIB_paramId:                             130
    GRIB_dataType:                            an
    GRIB_numberOfPoints:                      2652
    GRIB_typeOfLevel:                         hybrid
    GRIB_stepUnits:                           1
    GRIB_stepType:                            instant
    ...                                       ...
    GRIB_pv:                                  [0.00000000e+00 2.00000000e+01 ...
    GRIB_shortName:                           t
    GRIB_units:                               K
    long_name:                                Temperature
    units:                                    K
    standard_name:                            air_temperature

In [3]:
temperature['longitude'] = temperature['longitude']-360 ## change longitude to -180 to 180 format
temperature

<xarray.DataArray 't' (valid_time: 2920, latitude: 34, longitude: 78)> Size: 31MB
[7743840 values with dtype=float32]
Coordinates:
  * valid_time  (valid_time) datetime64[ns] 23kB 2010-01-01 ... 2010-12-31T21...
  * latitude    (latitude) float64 272B 49.25 48.5 47.75 ... 26.0 25.25 24.5
  * longitude   (longitude) float64 624B -124.8 -124.1 -123.3 ... -67.85 -67.1
Attributes: (12/31)
    GRIB_paramId:                             130
    GRIB_dataType:                            an
    GRIB_numberOfPoints:                      2652
    GRIB_typeOfLevel:                         hybrid
    GRIB_stepUnits:                           1
    GRIB_stepType:                            instant
    ...                                       ...
    GRIB_pv:                                  [0.00000000e+00 2.00000000e+01 ...
    GRIB_shortName:                           t
    GRIB_units:                               K
    long_name:                                Temperature
    units:                                    K
    standard_name:                            air_temperature

In [4]:
temperature.to_dataframe()

t
valid_time          latitude longitude            
2010-01-01 00:00:00 49.25    -124.85    275.571350
                             -124.10    275.530334
                             -123.35    275.405334
                             -122.60    274.099670
                             -121.85    271.778381
...                                            ...
2010-12-31 21:00:00 24.50    -70.10     295.699036
                             -69.35     295.426575
                             -68.60     295.209778
                             -67.85     295.077942
                             -67.10     294.934387

[7743840 rows x 1 columns]

In [5]:
## Lets load the county shapefile

SHAPE_PATH = os.path.join('../Shapefiles','county_shapefiles','2010_county_shapefile','gz_2010_us_050_00_500k.shp')
county_gdf = gpd.read_file(SHAPE_PATH)

county_gdf.head()

,GEO_ID,STATE,COUNTY,NAME,LSAD,CENSUSAREA,geometry
0,0500000US01029,01,029,Cleburne,County,560.100,"POLYGON ((-85.38872 33.91304, -85.38088 33.873..."
1,0500000US01031,01,031,Coffee,County,678.972,"POLYGON ((-86.03044 31.61894, -86.00408 31.619..."
2,0500000US01037,01,037,Coosa,County,650.926,"POLYGON ((-86.00928 33.10164, -86.00917 33.090..."
3,0500000US01039,01,039,Covington,County,1030.456,"POLYGON ((-86.34851 30.99434, -86.35023 30.994..."
4,0500000US01041,01,041,Crenshaw,County,608.840,"POLYGON ((-86.14699 31.68045, -86.14711 31.663..."


In [6]:
geometry_column=county_gdf['geometry'] ## get the geometry column
geometry_column

0       POLYGON ((-85.38872 33.91304, -85.38088 33.873...
1       POLYGON ((-86.03044 31.61894, -86.00408 31.619...
2       POLYGON ((-86.00928 33.10164, -86.00917 33.090...
3       POLYGON ((-86.34851 30.99434, -86.35023 30.994...
4       POLYGON ((-86.14699 31.68045, -86.14711 31.663...
                              ...                        
3216    POLYGON ((-66.90748 18.25314, -66.90739 18.253...
3217    POLYGON ((-66.37968 17.94398, -66.38029 17.943...
3218    MULTIPOLYGON (((-66.13957 18.46232, -66.13956 ...
3219    POLYGON ((-66.02917 18.37590, -66.02828 18.376...
3220    POLYGON ((-66.85229 17.95500, -66.85280 17.955...
Name: geometry, Length: 3221, dtype: geometry

In [7]:
lat_lon=geometry_column.get_coordinates(index_parts=True) ## get lat and lon from the geometry
lat_lon

x          y
0    0   -85.388717  33.913044
     1   -85.380885  33.873508
     2   -85.379455  33.866291
     3   -85.377426  33.856047
     4   -85.376403  33.850656
...             ...        ...
3220 202 -66.833718  17.989763
     203 -66.835282  17.988274
     204 -66.835429  17.986323
     205 -66.836682  17.965971
     206 -66.852288  17.955004

[837022 rows x 2 columns]

In [8]:
max_temp=305.372  ## threshold temperature expressed in Kelvin, which is 90 Fahrenheit
min_temp=273.15 ## threshold temperature expressed in Kelvin, which is 0 degree celsius

### fraction of time temperature was above max_temp

In [9]:
def single_county(var):
    
    ''' Function to interpolate the values in a single county by taking a sample of 10 latitudes and
         10 longitudes. The interpolation done by xarray will give a set of 100 points. A single county
         will consist of 365 X 8 = 2920 data. From this set we find the fraction of time a particular
         county was above a certain threshold or below a threshold.
        Args:
        --------
             var (int): The index of the county in the shapefile.

        Returns:
        --------
            temp_above_df: Dataframe consisting the fraction of time in a county above a certain threshold.
    '''    
    
    
    longitude= lat_lon.loc[(var), 'x']  # get the longitude
    longitude=longitude.sample(n=10)    # randomly select 10 points

    latitude= lat_lon.loc[(var), 'y']  # get the latitude
    latitude=latitude.sample(n=10)    ## randomly select 10 points

    lat_list=latitude.tolist()
    lon_list=longitude.tolist()
    
    year_avg_finer= temperature.interp(longitude=lon_list, latitude=lat_list)
    

    summary = year_avg_finer.groupby("valid_time").mean(["latitude", "longitude"])
    df=summary.to_dataframe()
    total_rows=len(df)

    c=0
    for i in range(0, total_rows):
        if df['t'].values[i] > max_temp :
            c=c+1
    ## convert the number to a fraction 

    frac_time=(c/total_rows)*100
    temp_above_df=pd.DataFrame({'Temp above threshold':[frac_time]})
    
    return temp_above_df

In [10]:
single_county(0)

,Temp above threshold
0,3.013699


In [11]:
%%time

## get the values for all counties

df_list_above=[]

for i in range(0,3221): ## A total of 3221 counties. index in shape file starts from 0 and ends in 3220
    try:
        df_list_above.append(single_county(i))
    except:
        pass

CPU times: total: 49min 22s
Wall time: 50min 15s


In [12]:
initial_df_above=pd.concat(df_list_above)
initial_df_above

,Temp above threshold
0,3.047945
0,4.041096
0,4.486301
0,4.143836
0,4.863014
...,...
0,0.000000
0,0.000000
0,0.000000
0,0.000000


### fraction of time temperature was below min temperature

In [13]:
def single_county(var):
    
    ''' Function to interpolate the values in a single county by taking a sample of 10 latitudes and
         10 longitudes. The interpolation done by xarray will give a set of 100 points. A single county
         will consist of 365 X 8 = 2920 data. From this set we find the fraction of time a particular
         county was above a certain threshold or below a threshold.
        Args:
        --------
             var (int): The index of the county in the shapefile.

        Returns:
        --------
            county_df: Dataframe consisting the fraction of time in a county below a certain threshold.
    '''    
    
    
    longitude= lat_lon.loc[(var), 'x']  # get the longitude
    longitude=longitude.sample(n=10)    # randomly select 10 points

    latitude= lat_lon.loc[(var), 'y']  # get the latitude
    latitude=latitude.sample(n=10)    ## randomly select 10 points

    lat_list=latitude.tolist()
    lon_list=longitude.tolist()
    
    year_avg_finer= temperature.interp(longitude=lon_list, latitude=lat_list)
    

    summary = year_avg_finer.groupby("valid_time").mean(["latitude", "longitude"])
    df=summary.to_dataframe()
    total_rows=len(df)

    c=0
    for i in range(0, total_rows):
        if df['t'].values[i] <= min_temp :
            c=c+1
    ## convert the number to a fraction 

    frac_time=(c/total_rows)*100
    county_df=pd.DataFrame({'Temp below threshold':[frac_time]})
    
    return county_df

In [14]:
single_county(0)

,Temp below threshold
0,7.739726


In [15]:
%%time

## get the values for all counties

df_list_below=[]

for i in range(0,3221): ## A total of 3221 counties. index in shape file starts from 0 and ends in 3220
    try:
        df_list_below.append(single_county(i))
    except:
        pass

CPU times: total: 49min 39s
Wall time: 50min 23s


In [16]:
initial_df_below=pd.concat(df_list_below)
initial_df_below

,Temp below threshold
0,7.842466
0,3.595890
0,5.890411
0,3.116438
0,3.835616
...,...
0,0.000000
0,0.000000
0,0.000000
0,0.000000


### feature engineering PM2.5

In [27]:
PATH_2=os.path.join('../Weather_Data','CAMS','2010','2010_single_level_34_variables.nc')
sl_34_variables = xr.open_dataset(PATH_2) ## multi level 26 variables
pm=sl_34_variables['pm2p5'] ## get only pm2.5 variable
pm

<xarray.DataArray 'pm2p5' (valid_time: 2920, latitude: 34, longitude: 78)> Size: 31MB
[7743840 values with dtype=float32]
Coordinates:
  * valid_time  (valid_time) datetime64[ns] 23kB 2010-01-01 ... 2010-12-31T21...
  * latitude    (latitude) float64 272B 49.25 48.5 47.75 ... 26.0 25.25 24.5
  * longitude   (longitude) float64 624B -124.8 -124.1 -123.3 ... -67.85 -67.1
Attributes: (12/33)
    GRIB_paramId:                             210073
    GRIB_dataType:                            an
    GRIB_numberOfPoints:                      2652
    GRIB_typeOfLevel:                         surface
    GRIB_stepUnits:                           1
    GRIB_stepType:                            instant
    ...                                       ...
    GRIB_units:                               kg m**-3
    long_name:                                Particulate matter d <= 2.5 um
    units:                                    kg m**-3
    standard_name:                            mass_concentration_of_pm2p5_amb...
    GRIB_number:                              0
    GRIB_surface:                             0.0

In [28]:
pm.to_dataframe()

pm2p5
valid_time          latitude longitude              
2010-01-01 00:00:00 49.25    -124.85    4.495104e-09
                             -124.10    7.791134e-09
                             -123.35    9.966738e-09
                             -122.60    8.202936e-09
                             -121.85    4.889633e-09
...                                              ...
2010-12-31 21:00:00 24.50    -70.10     3.764598e-09
                             -69.35     4.265615e-09
                             -68.60     4.915968e-09
                             -67.85     5.530126e-09
                             -67.10     6.248644e-09

[7743840 rows x 1 columns]

In [40]:
pm_threshold=9*10**(-9)
pm_threshold

9.000000000000001e-09

In [42]:
def single_county(var):
    
    ''' Function to interpolate the values in a single county by taking a sample of 10 latitudes and
         10 longitudes. The interpolation done by xarray will give a set of 100 points. A single county
         will consist of 365 X 8 = 2920 data. From this set we find the fraction of time a particular
         county was above a certain threshold or below a threshold.
        Args:
        --------
             var (int): The index of the county in the shapefile.

        Returns:
        --------
            pm_above_df: Dataframe consisting the fraction of time in a county above a certain threshold.
    '''    
    
    
    longitude= lat_lon.loc[(var), 'x']  # get the longitude
    longitude=longitude.sample(n=10)    # randomly select 10 points

    latitude= lat_lon.loc[(var), 'y']  # get the latitude
    latitude=latitude.sample(n=10)    ## randomly select 10 points

    lat_list=latitude.tolist()
    lon_list=longitude.tolist()
    
    year_avg_finer= pm.interp(longitude=lon_list, latitude=lat_list)
    

    summary = year_avg_finer.groupby("valid_time").mean(["latitude", "longitude"])
    df=summary.to_dataframe()
    total_rows=len(df)

    c=0
    for i in range(0, total_rows):
        if df['pm2p5'].values[i] > pm_threshold :
            c=c+1
    ## convert the number to a fraction 

    frac_time=(c/total_rows)*100
    pm_above_df=pd.DataFrame({'PM2.5 above threshold':[frac_time]})
    
    return pm_above_df

In [53]:
single_county(0)

,PM2.5 above threshold
0,70.890411


In [ ]:
%%time

## get the values for all counties

pm_above=[]

for i in range(0,3221): ## A total of 3221 counties. index in shape file starts from 0 and ends in 3220
    try:
        pm_above.append(single_county(i))
    except:
        pass

In [ ]:
pm_df_above=pd.concat(pm_above)
pm_df_above

### Lets begin merging

In [23]:
## lets first create a fips column in the original shape file

## lets create the fips column in the dataframe which is the sum of state and county both as dtype string.
## Note: not a numerical sum
## This step is necessary because there can be several counties with idential names. 

county_gdf['fips']=county_gdf['STATE'] + county_gdf['COUNTY']
county_gdf.dtypes
## convert the data type
county_gdf['fips']=county_gdf['fips'].astype(str).astype(int)
county_gdf.dtypes
county_gdf = county_gdf.drop([ 'GEO_ID','CENSUSAREA','STATE','COUNTY','LSAD'], axis=1)
county_gdf

,NAME,geometry,fips
0,Cleburne,"POLYGON ((-85.38872 33.91304, -85.38088 33.873...",1029
1,Coffee,"POLYGON ((-86.03044 31.61894, -86.00408 31.619...",1031
2,Coosa,"POLYGON ((-86.00928 33.10164, -86.00917 33.090...",1037
3,Covington,"POLYGON ((-86.34851 30.99434, -86.35023 30.994...",1039
4,Crenshaw,"POLYGON ((-86.14699 31.68045, -86.14711 31.663...",1041
...,...,...,...
3216,San Sebastián,"POLYGON ((-66.90748 18.25314, -66.90739 18.253...",72131
3217,Santa Isabel,"POLYGON ((-66.37968 17.94398, -66.38029 17.943...",72133
3218,Toa Baja,"MULTIPOLYGON (((-66.13957 18.46232, -66.13956 ...",72137
3219,Trujillo Alto,"POLYGON ((-66.02917 18.37590, -66.02828 18.376...",72139
